<a href="https://colab.research.google.com/github/MMathisLab/DeepLabCut/blob/master/Colab_DEMO_mouse_openfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab Demo on topview mouse data
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless3D  pose estimation during behavior across species. Nature Protocols, 2019

This notebook demonstrates the necessary steps to use DeepLabCut on our demo data. We provide labeled mouse data from Mathis et al, 2018 Nature Neuroscience.

This demo notebook mostly shows the most simple code to train and evaluate your model, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to use the cloud to:
-load demo data
- create a training set
- train a network
- evaluate a network
- analyze a novel video



##First,  let's look at info about the Colab Environment (and decide, GPU or TPU):

First, go to "Runtime" ->"change runtime type"->select "Python3", and then decide, "GPU or TPU!"" (GPUs are faster)


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


If you want to use a GPU:

In [2]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [3]:
#let's make sure we see a GPU:
tf.test.gpu_device_name()

#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

'/device:GPU:0'

If you selected to use a TPU: 

In [0]:
#let's make sure we see a TPU:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

## Link your Google Drive (with our demo data):

### First, download this project folder and place into your Google Drive: 
https://github.com/AlexEMG/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30

In [4]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
#(this will take a few minutes to install all the dependences!)

!pip install deeplabcut

    100% |████████████████████████████████| 143kB 8.2MB/s 
    100% |████████████████████████████████| 276kB 26.2MB/s 
    100% |████████████████████████████████| 215kB 28.3MB/s 
    100% |████████████████████████████████| 737kB 21.3MB/s 
    100% |████████████████████████████████| 552kB 25.8MB/s 
    100% |████████████████████████████████| 25.3MB 1.8MB/s 
    100% |████████████████████████████████| 51kB 22.7MB/s 
    100% |████████████████████████████████| 26.2MB 1.8MB/s 
    100% |████████████████████████████████| 4.9MB 8.1MB/s 
    100% |████████████████████████████████| 3.3MB 8.3MB/s 
    100% |████████████████████████████████| 7.4MB 6.8MB/s 
    100% |████████████████████████████████| 716kB 23.4MB/s 
    100% |████████████████████████████████| 2.0MB 15.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 wh

In [0]:
#GUIs don't work on the cloud, so we will supress wxPython: 
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

In [7]:
#Be sure you have a copy of your project folder in your Google Drive. Then, let's go to your deeplabcut folder!
%cd /content/drive/My Drive/DeepLabCut

/content/drive/My Drive/DeepLabCut


In [10]:
import deeplabcut

#sometimes the backend kernel is defaulting to something other than what we want
#if you see the warnings, you can re-run this cell and they go way, as we re-set the backend! :) 

#possible warning--> UserWarning: This call to matplotlib.use() has no effect because the backend has already
#been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot, or matplotlib.backends is imported for the first time.

Project loaded in colab-mode. Apparently Colab has trouble loading statsmodels, so the smooting & outlier frame extraction is disabled. Sorry!


In [13]:
#These are some Colab specific work-arounds, but they work! (typically not required, as they are installed with "pip install deeplabcut")

!pip install Pillow==4.0.0

from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions


    100% |████████████████████████████████| 5.6MB 8.2MB/s 
torchvision 0.2.2.post3 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
scikit-image 0.14.2 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 6.0.0
    Uninstalling Pillow-6.0.0:
      Successfully uninstalled Pillow-6.0.0


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!


In [0]:
#create a path variable that links to your google drive copy:
path_config_file = '/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/config.yaml' #change to yours!

In [15]:
# Loading example data set:
import os
#path_config_file = os.path.join(os.getcwd(),'openfield-Pranav-2018-10-30/config.yaml')
deeplabcut.load_demo_data(path_config_file)

Loaded, now creating training data...
/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [0]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/content/drive/My '
                 'Drive/DeepLabCut/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1000,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/Documentation_data-openfield_95shuffle1.pickle',
 'minsize': 100,
 'm

Instructions for updating:
Colocations handled automatically by placer.


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:209: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


...

From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/DeepLabCut/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /content/drive/My Drive/DeepLabCut/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Display_iters overwritten as 10
Save_iters overwritten as 500
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight

iteration: 10 loss: 0.2833 lr: 0.005
iteration: 20 loss: 0.0537 lr: 0.005
iteration: 30 loss: 0.0435 lr: 0.005
iteration: 40 loss: 0.0376 lr: 0.005


We recommend you run this for ~10,000 iterations, just as a demo. This should take around 90 min. Note, that **when you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [0]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/Documentation_data-openfield_95shuffle1.pickle',
 'min_input_size': 64,
 'minsize': 1

Running  DeepCut_resnet50_openfieldOct30shuffle1_3500  with # of trainingiterations: 3500
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot-3500


Restoring parameters from /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot-3500
0it [00:00, ?it/s]

Analyzing data...


116it [00:13,  8.52it/s]


Done and results stored for snapshot:  snapshot-3500
Results for 3500  training iterations: 95 1 train error: 3.11 pixels. Test error: 3.04  pixels.
With pcutoff of 0.1  train error: 3.11 pixels. Test error: 3.04 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


**Check the images**:
You can go look in the newly created "evalutaion-results" folder at the images. At around 3500 iterations, the error is ~3 pixels (but this can vary on how your demo data was split for training)

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

**On the demo data, this should take around ~ 3 min! **

In [0]:
videofile_path = ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['snout', 'leftear', 'rightear', 'tailbase'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/openfield_Pranav95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.6/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_openfieldOct30/Documentation_data-openfield_95shuffle1.pickle',
 'min_input_size': 64,
 'minsize': 1

Using snapshot-3500 for model /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot-3500


Restoring parameters from /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/dlc-models/iteration-0/openfieldOct30-trainset95shuffle1/train/snapshot-3500


Starting to analyze %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4


  0%|          | 0/2330 [00:00<?, ?it/s]

Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


2346it [02:50, 13.69it/s]                          

Saving results in /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

Starting %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4']
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.


  1%|          | 17/2330 [00:00<00:14, 163.34it/s]

False 0 640 0 480
2330
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330 with cropped frame dimensions:  640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [00:14<00:00, 158.64it/s]


## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
deeplabcut.plot_trajectories(path_config_file,videofile_path)

/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4
Starting %  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos ['/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4']
/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos  already exists!
/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/plot-poses  already exists!
/content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/plot-poses/m3v1mp4  already exists!
Loading  /content/drive/My Drive/DeepLabCut/examples/openfield-Pranav-2018-10-30/videos/m3v1mp4.mp4 and data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plots created! Please check the directory "plot-poses" within the video directory
